In [1]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq 
import os
from dotenv import load_dotenv
import time
import pprint
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.document_loaders import WebBaseLoader
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool
from langchain_core.messages import HumanMessage

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_KEY')
os.environ["GROQ_API_KEY"] = os.getenv('GROQ_API_KEY')

os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')
os.environ["GOOGLE_CSE_ID"] = os.getenv('GOOGLE_CSE_ID')


llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

In [2]:
# response = llm.invoke([HumanMessage(content="¿Que es LangSmith?")])
response = llm.invoke("¿Que es LangSmith?")
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: LangSmith es una empresa de traducción y localización de idiomas que ofrece servicios lingüísticos profesionales para empresas y particulares. Ofrecen traducciones en una amplia variedad de idiomas y sectores, así como servicios de interpretación, transcripción y subtitulación. También proporcionan servicios de localización de software, páginas web y contenido multimedia para adaptarlo a diferentes mercados y culturas.
ToolCalls: []


In [3]:
# llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
# respuesta = llm.predict("¿Que es LangSmith?")
# pprint.pprint(respuesta)

## Creamos la herramienta

In [3]:
search = GoogleSearchAPIWrapper()

google_search = Tool(
    name="google_search",
    description=("Usa esta herramienta cuando tengas que responder a una pregunta. Esta herramienta te permitirá realizar una busqueda en google para obtener el contexto para responder correctametne a la pregunta."
                "Busca por palabras clave"),
    func=search.run,
)

f:\GitHub\langchain_webinar\myenv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 0.3.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import GoogleSearchAPIWrapper`.
  warn_deprecated(


In [4]:
tools = [google_search]

## Le decimos como queremos que actue

In [5]:
from langchain_core.prompts import PromptTemplate

TEMPLATE = """

The final answer has to be in spanish.
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

Final Answer: the final answer to the original input question

Begin!

Question: {input}

Thought:{agent_scratchpad}

"""

prompt = PromptTemplate(template=TEMPLATE, input_variables=["tools", "input", "agent_scratchpad", "tool_names"])

In [6]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_openai import OpenAI

# prompt = hub.pull("hwchase17/react")
# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

In [7]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [8]:
from langchain.globals import set_verbose, set_debug

set_debug(True)
agent_executor.invoke({"input": "¿Que es LangSmith?"})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "¿Que es LangSmith?"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": ""
}
[chain

{'input': '¿Que es LangSmith?',
 'output': 'LangSmith es una herramienta que permite a los programadores experimentar con la inteligencia artificial y prototipar aplicaciones en minutos.'}

<img src="img/5.png" width="500" height="340"> 

In [9]:
agent_executor.invoke({"input": "¿Que es wikipedia?"})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "¿Que es wikipedia?"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": ""
}
[chain

{'input': '¿Que es wikipedia?',
 'output': 'Wikipedia es una enciclopedia en línea gratuita, creada y editada por voluntarios de todo el mundo y alojada por la Fundación Wikimedia.'}